In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
from fastai.tabular import *

In [ ]:
df = pd.read_csv('../input/train_V2.csv')

In [ ]:
path = Path('../')

In [ ]:
df_test = pd.read_csv('../input/test_V2.csv')

In [ ]:
df_output = pd.read_csv('../input/sample_submission_V2.csv')

In [ ]:
df[df.winPlacePerc.isnull()]

In [ ]:
df = df.drop([2744604], axis=0)

In [ ]:
dep_var = 'winPlacePerc'
cat_names = ['Id', 'groupId', 'matchId', 'matchType']
cont_names = ['assists', 'boosts', 'damageDealt', 'DBNOs',
              'headshotKills', 'heals', 'killPlace', 'killPoints', 'kills', 'killStreaks', 'longestKill', 'matchDuration', 'maxPlace', 'numGroups',
              'rankPoints', 'revives', 'rideDistance', 'roadKills', 'swimDistance', 'teamKills', 'vehicleDestroys', 'walkDistance', 'weaponsAcquired',
              'winPoints']
procs = [FillMissing, Categorify, Normalize]

In [ ]:
test = TabularList.from_df(df.iloc[800:1000].copy(), path=path, cat_names=cat_names, cont_names=cont_names, procs=procs)

In [ ]:
data = (TabularList.from_df(df.loc[:500000], path=path, cat_names=cat_names, cont_names=cont_names, procs=procs)
                           .random_split_by_pct(valid_pct=0.2)
                           .label_from_df(cols=dep_var)
                           
                           .databunch(bs=32))

In [ ]:
learn = tabular_learner(data, layers=[200,100], metrics=mse)

In [ ]:
learn.fit(1)

In [ ]:
cat, idx, output = learn.predict(df.loc[600003])

In [ ]:
l = []
for x in range(df_test.Id.count()):
    cat, idx, output = learn.predict(df_test.loc[x])
    df_output.loc[x, 'winPlacePerc'] = str(float(output))
print("done")

In [ ]:
data_to_submit = pd.DataFrame({
    'Id':df_output['Id'],
    'winPlacePerc':df_output['winPlacePerc']
})


In [ ]:
data_to_submit.to_csv('csv_to_submit.csv', index = False)